In [102]:
import pickle
import warnings

from collections import Counter
from copy import deepcopy
from datetime import datetime
from typing import Dict

import numpy as np
import pandas as pd
import scipy as sp

from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender, ItemItemRecommender
from IPython.display import display
from rectools.metrics.classification import Accuracy, Precision, Recall
from rectools.metrics.novelty import MeanInvUserFreq
from rectools.metrics.serendipity import Serendipity
from rectools.metrics.ranking import MAP, NDCG
from rectools.metrics import calc_metrics
from rectools import Columns
from rectools.models.popular import PopularModel

from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset, Interactions

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 40)
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

## Возьмем класс `UserKnn` из семинара

In [51]:
class UserKnn():
    """Class for fit-perdict UserKNN model 
       based on ItemKNN model from implicit.nearest_neighbours
    """

    def __init__(self, model: ItemItemRecommender, N_users: int = 50):
        self.N_users = N_users
        self.model = model
        self.is_fitted = False

    def get_mappings(self, train):
        self.users_inv_mapping = dict(enumerate(train['user_id'].unique()))
        self.users_mapping = {v: k for k, v in self.users_inv_mapping.items()}

        self.items_inv_mapping = dict(enumerate(train['item_id'].unique()))
        self.items_mapping = {v: k for k, v in self.items_inv_mapping.items()}

    def get_matrix(self, df: pd.DataFrame, 
                   user_col: str = 'user_id', 
                   item_col: str = 'item_id', 
                   weight_col: str = None, 
                   users_mapping: Dict[int, int] = None, 
                   items_mapping: Dict[int, int] = None):

        if weight_col:
            weights = df[weight_col].astype(np.float32)
        else:
            weights = np.ones(len(df), dtype=np.float32)

        self.interaction_matrix = sp.sparse.coo_matrix((
            weights, 
            (
                df[item_col].map(self.items_mapping.get),
                df[user_col].map(self.users_mapping.get)
            )
            ))

        self.watched = df\
            .groupby(user_col, as_index=False)\
            .agg({item_col: list})\
            .rename(columns={user_col: 'sim_user_id'})

        return self.interaction_matrix

    def idf(self, n: int, x: float):
        return np.log((1 + n) / (1 + x) + 1)

    def _count_item_idf(self, df: pd.DataFrame):
        item_cnt = Counter(df['item_id'].values)
        item_idf = pd.DataFrame.from_dict(item_cnt, orient='index', 
                                          columns=['doc_freq']).reset_index()
        item_idf['idf'] = item_idf['doc_freq'].apply(lambda x: self.idf(self.n, x))
        self.item_idf = item_idf 

    def fit(self, train: pd.DataFrame):
        self.user_knn = self.model
        self.get_mappings(train)
        self.weights_matrix = self.get_matrix(train, 
                                              users_mapping=self.users_mapping, 
                                              items_mapping=self.items_mapping)

        self.n = train.shape[0]
        self._count_item_idf(train)

        self.user_knn.fit(self.weights_matrix)
        self.is_fitted = True

    def _generate_recs_mapper(self, model: ItemItemRecommender, user_mapping: Dict[int, int], 
                              user_inv_mapping: Dict[int, int], N: int):
        def _recs_mapper(user):
            user_id = self.users_mapping[user]
            users, sim = model.similar_items(user_id, N=N)
            return [self.users_inv_mapping[user] for user in users], sim
        return _recs_mapper

    def predict(self, test: pd.DataFrame, N_recs: int = 10):

        if not self.is_fitted:
            raise ValueError("Please call fit before predict")

        mapper = self._generate_recs_mapper(
            model=self.user_knn, 
            user_mapping=self.users_mapping,
            user_inv_mapping=self.users_inv_mapping,
            N=self.N_users
        )

        recs = pd.DataFrame({'user_id': test['user_id'].unique()})
        recs['sim_user_id'], recs['sim'] = zip(*recs['user_id'].map(mapper))
        recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

        recs = recs[~(recs['user_id'] == recs['sim_user_id'])]\
            .merge(self.watched, on=['sim_user_id'], how='left')\
            .explode('item_id')\
            .sort_values(['user_id', 'sim'], ascending=False)\
            .drop_duplicates(['user_id', 'item_id'], keep='first')\
            .merge(self.item_idf, left_on='item_id', right_on='index', how='left')

        recs['score'] = recs['sim'] * recs['idf']
        recs = recs.sort_values(['user_id', 'score'], ascending=False)
        recs['rank'] = recs.groupby('user_id').cumcount() + 1 
        return recs[recs['rank'] <= N_recs][['user_id', 'item_id', 'score', 'rank']]

## Берем датасет

In [52]:
# Взаимодействия
interactions_df = pd.read_csv('../data/interactions.csv', parse_dates=["last_watch_dt"])

interactions_df.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, 
    inplace=True)

# Пользователи
users = pd.read_csv('../data/users.csv')

# Айтемы
items = pd.read_csv('../data/items.csv')

In [53]:
interactions = Interactions(interactions_df)
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,"4,250.000000",72.000000
1,699317,1659,2021-05-29,"8,317.000000",100.000000
2,656683,7107,2021-05-09,10.000000,0.000000
3,864613,7638,2021-07-05,"14,483.000000",100.000000
4,964868,9506,2021-04-30,"6,725.000000",100.000000


In [54]:
def headtail(df):
    """
    Вспомогательная функция для вывода первых 5 и последних 5 записей датасета
    """
    return pd.concat([df.head(), df.tail()])

headtail(items)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,"2,002.000000","драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.000000,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,"2,014.000000","зарубежные, приключения, комедии",США,NaN,16.000000,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,"2,011.000000","криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.000000,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,"2,015.000000","драмы, зарубежные, мелодрамы",Великобритания,NaN,16.000000,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,"1,978.000000","драмы, спорт, советские, мелодрамы",СССР,NaN,12.000000,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
15958,6443,series,Полярный круг,Arctic Circle,"2,018.000000","драмы, триллеры, криминал","Финляндия, Германия",NaN,16.000000,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,"2,020.000000","драмы, боевики",Россия,0.000000,18.000000,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,"2,017.000000","драмы, триллеры, криминал",Россия,0.000000,18.000000,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,"2,019.000000","драмы, спорт, криминал",Россия,0.000000,18.000000,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,"2,019.000000",комедии,Россия,0.000000,16.000000,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы...","Гоша, 2019, Россия"


## Инициализируем сплиттер, словари с моделями и метриками

In [55]:
splitter = TimeRangeSplitter(test_size="7D", 
                             n_splits=3, 
                             filter_cold_users=True,
                             filter_cold_items=True,
                             filter_already_seen=True)

In [56]:
splitter.get_test_fold_borders(interactions=interactions)

[(Timestamp('2021-08-02 00:00:00', freq='7D'),
  Timestamp('2021-08-09 00:00:00', freq='7D')),
 (Timestamp('2021-08-09 00:00:00', freq='7D'),
  Timestamp('2021-08-16 00:00:00', freq='7D')),
 (Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

Я пытался еще взять ItemItemRecommender, но он у меня падал с ошибкой Buffer dispatch, которую я не нашел, как исправить

In [57]:
models_dict = {
    "knn_Cosine": UserKnn(CosineRecommender(K=10), N_users=50),
    "knn_TFIDF": UserKnn(TFIDFRecommender(K=10), N_users=50),
    "knn_BM25": UserKnn(BM25Recommender(K=10), N_users=50)
}

metrics_dict = {
    'MAP_@10': MAP(k=10),
    "NDCG_@10": NDCG(k=10),
    "Accuracy_@10": Accuracy(k=10),
    "Precision_@10": Precision(k=10),
    "Recall_@10": Recall(k=10),
    "Serendipity_@10": Serendipity(k=10),
    'MeanInvUserFreq_@10': MeanInvUserFreq(k=10)
}

## Берем функцию кросс-валидации из прошлой домашки, меняем ее совсем немного под работу с классом `UserKnn`

In [58]:
def cross_validation(models: Dict,
                     metrics: Dict,
                     splitter: TimeRangeSplitter,
                     k: int,
                     interactions: Interactions,
                     log_model: bool=False) -> pd.DataFrame:
    results = []
    models_logs = []
    
    folds = splitter.split(interactions)

    for train_indices, test_indices, test_borders in folds:
        train_df = interactions.df.iloc[train_indices]  # pd.DataFrame (test) (не rectools.Dataset!!!)

        test_df = interactions.df.iloc[test_indices][["user_id", "item_id"]]  # pd.DataFrame (test)
        
        users = np.unique(test_df["user_id"])  # users & items
        items = interactions.df.iloc[train_indices]["item_id"].unique()

        for model_name, model in models.items():
            model_info = {}
            cur_model_log = {}
            
            print(f'Model: {model_name}, N_split: {test_borders["i_split"]}, Test borders: {test_borders["start"].strftime("%Y-%m-%d"), test_borders["end"].strftime("%Y-%m-%d")}')
            model = deepcopy(model)
            
            model.fit(train_df)  # fit & predict
            if log_model:  # логгирование времени модели
                cur_model_log['model_name'] = model_name
                cur_model_log['n_split'] = test_borders["i_split"]
                cur_model_log['time_fitted'] = datetime.now()
                models_logs.append(cur_model_log)

            recommendations = model.predict(train_df)

            metric_values = calc_metrics(  # metrics
                metrics=metrics,
                reco=recommendations,
                interactions=test_df,
                prev_interactions=interactions.df.iloc[train_indices],
                catalog=items
            )

            model_info["model_name"] = model_name
            model_info["n_split"] = test_borders["i_split"]
            model_info |= metric_values

            results.append(model_info)
    
    if not log_model:
        return pd.DataFrame(results).groupby(["model_name"]).mean().reset_index().drop(columns=["n_split"], axis=1)
    return pd.DataFrame(results).groupby(["model_name"]).mean().reset_index().drop(columns=["n_split"], axis=1), models_logs

In [59]:
df, models_logs = cross_validation(models=models_dict,
                      metrics=metrics_dict,
                      k=10,
                      interactions=interactions,
                      splitter=splitter,
                      log_model=True)

Model: knn_Cosine, N_split: 0, Test borders: ('2021-08-02', '2021-08-09')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 797423/797423 [12:04<00:00, 1100.63it/s]


Model: knn_TFIDF, N_split: 0, Test borders: ('2021-08-02', '2021-08-09')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 797423/797423 [12:00<00:00, 1106.72it/s]


Model: knn_BM25, N_split: 0, Test borders: ('2021-08-02', '2021-08-09')


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906071/906071 [16:12<00:00, 931.24it/s]


Model: knn_TFIDF, N_split: 2, Test borders: ('2021-08-16', '2021-08-23')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906071/906071 [16:10<00:00, 933.52it/s]


Model: knn_BM25, N_split: 2, Test borders: ('2021-08-16', '2021-08-23')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 906071/906071 [16:10<00:00, 934.07it/s]


## Исследуем метрики!

- Нас интересует метрики MAP_@10, так как именно по ней формируется лидерборд
- Можем заметить, что лучше всех справился knn+TFIDFRecommender
- В принципе и почти по всем остальным метрикам `TFIDF` лучше всех, __так что возьмем эту модель для решения задания__

In [60]:
df

,model_name,Accuracy_@10,Precision_@10,Recall_@10,NDCG_@10,MAP_@10,MeanInvUserFreq_@10,Serendipity_@10
0,knn_BM25,0.999180,0.003538,0.016029,0.002917,0.003164,7.477723,0.000011
1,knn_Cosine,0.999181,0.004142,0.020041,0.003442,0.003958,6.384824,0.000006
2,knn_TFIDF,0.999183,0.005890,0.029503,0.004799,0.005659,6.494855,0.000007


In [61]:
models_logs

[{'model_name': 'knn_Cosine',
  'n_split': 0,
  'time_fitted': datetime.datetime(2023, 11, 28, 1, 9, 49, 288780)},
 {'model_name': 'knn_TFIDF',
  'n_split': 0,
  'time_fitted': datetime.datetime(2023, 11, 28, 1, 23, 22, 646887)},
 {'model_name': 'knn_BM25',
  'n_split': 0,
  'time_fitted': datetime.datetime(2023, 11, 28, 1, 36, 56, 719738)},
 {'model_name': 'knn_Cosine',
  'n_split': 1,
  'time_fitted': datetime.datetime(2023, 11, 28, 1, 52, 46, 2609)},
 {'model_name': 'knn_TFIDF',
  'n_split': 1,
  'time_fitted': datetime.datetime(2023, 11, 28, 2, 8, 30, 717594)},
 {'model_name': 'knn_BM25',
  'n_split': 1,
  'time_fitted': datetime.datetime(2023, 11, 28, 2, 24, 16, 774619)},
 {'model_name': 'knn_Cosine',
  'n_split': 2,
  'time_fitted': datetime.datetime(2023, 11, 28, 2, 42, 19, 450289)},
 {'model_name': 'knn_TFIDF',
  'n_split': 2,
  'time_fitted': datetime.datetime(2023, 11, 28, 3, 0, 15, 596325)},
 {'model_name': 'knn_BM25',
  'n_split': 2,
  'time_fitted': datetime.datetime(2023,

## Обучим лучшую модель

In [73]:
knn_tfidf = UserKnn(model=TFIDFRecommender(K=10), N_users=50)

In [75]:
knn_tfidf.fit(interactions_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 962179/962179 [18:44<00:00, 855.58it/s]


Посмотрим, как работает модель

In [78]:
max_date = interactions_df['datetime'].max()

test = interactions_df[(interactions_df['datetime'] >= max_date - pd.Timedelta(days=7))]

In [79]:
test

,user_id,item_id,datetime,weight,watched_pct
9,203219,13582,2021-08-22,"6,975.000000",100.000000
22,505244,15297,2021-08-15,"15,991.000000",63.000000
54,200197,9335,2021-08-16,83.000000,2.000000
64,73446,14488,2021-08-19,"6,011.000000",100.000000
65,125519,4583,2021-08-22,3.000000,0.000000
...,...,...,...,...,...
5476188,590892,8618,2021-08-21,"1,335.000000",23.000000
5476191,857162,12360,2021-08-16,11.000000,0.000000
5476201,273558,10605,2021-08-21,"34,030.000000",100.000000
5476248,697262,15297,2021-08-20,"18,307.000000",63.000000


In [81]:
recommendations = knn_tfidf.predict(test)

In [93]:
knn_recs = recommendations.groupby(['user_id']).agg({'item_id': lambda x: x.tolist()}).reset_index().set_index('user_id').to_dict()['item_id']

Не у всех пользователей есть 10 рекомендаций. Дополним им рекомендации до 10 самыми популярными айтемами

## Используем модель популярных айтемов

In [85]:
pop_model = PopularModel()

dataset = Dataset.construct(interactions_df=interactions_df)

pop_model.fit(dataset=dataset)

In [88]:
pop_recommendations = pop_model.recommend(
                users=test['user_id'].unique(),
                dataset=dataset,
                k=10,
                filter_viewed=True
            )

In [94]:
pop_recs = pop_recommendations.groupby(['user_id']).agg({'item_id': lambda x: x.tolist()}).reset_index().set_index('user_id').to_dict()['item_id']

## Напишем функцию для того, чтобы юзеру рекомендовало 10 айтемов

Заодно это ляжет в основу нашей функции для онлайн рекомендаций

In [168]:
def get_online_recs_for_user(knn_model,
                            pop_model,
                             pop_model_dataset,
                            user_id: int):
    if user_id in knn_model.users_mapping:
        knn_recommendations = knn_model.predict(pd.DataFrame([user_id], columns=['user_id']))
        knn_recommendations = knn_recommendations['item_id'].tolist()
    else:
        knn_recommendations = []


    if len(knn_recommendations) < 10:
        pop_recommendations = pop_model.recommend(
                users=[user_id],
                dataset=pop_model_dataset,
                k=10,
                filter_viewed=True
            )['item_id'].tolist()

        return {user_id: pd.unique(np.concatenate([knn_recommendations, pop_recommendations]))[:10].tolist()}

    return {user_id: knn_recommendations}


Примеры рекомендаций голой knn_tfidf:

In [169]:
knn_recs[1]

[3669, 10440]

In [170]:
knn_recs[12]

[846]

### Примеры онлайн рекомендаций вместе с PopularModel

In [171]:
get_online_recs_for_user(knn_model=knn_tfidf,
                         pop_model=pop_model,
                         pop_model_dataset=dataset,
                         user_id=1)

{1: [3669, 10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142]}

In [172]:
get_online_recs_for_user(knn_model=knn_tfidf,
                         pop_model=pop_model,
                         pop_model_dataset=dataset,
                         user_id=12)

{12: [846, 10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142]}

## Теперь сохраним нужные переменные в пиклы, чтобы потом использовать в онлайн рекомендациях

In [174]:
with open("../service/recsys_models/popular_model_20231128.pkl", "wb") as f:
    pickle.dump(pop_model, f)

with open("../service/recsys_models/knn_tfidf_model_20231127.pkl", "wb") as f:
    pickle.dump(knn_tfidf, f)

with open("../service/recsys_models/dataset_for_pop_model_20231128.pkl", "wb") as f:
    pickle.dump(dataset, f)

In [182]:
offline_recs = {}

for user in test['user_id'].unique():
    offline_recs[user] = get_online_recs_for_user(knn_model=knn_tfidf,
                         pop_model=pop_model,
                         pop_model_dataset=dataset,
                         user_id=user)

In [188]:
restructured_offline_recs = {user_id: offline_recs[user_id][user_id] for user_id in offline_recs}

In [192]:
with open("../service/recsys_models/knn_tfidf_model_offline_recos_20231128.pkl", "wb") as f:
    pickle.dump(restructured_offline_recs, f)

## С онлайном разобрались (подробнее в пулл реквесте). Теперь займемся оффлайн рекомендациями

### Посчитаем рекомендации для всех пользователей из датасета

In [206]:
knn_recs_for_all_users = knn_tfidf.predict(interactions_df)

In [217]:
knn_recs_for_all_users = knn_recs_for_all_users.groupby(['user_id']).agg({'item_id': lambda x: x.tolist()}).reset_index().set_index('user_id').to_dict()['item_id']

In [272]:
pop_recs_for_all_users = pop_model.recommend(
                users=interactions_df['user_id'].unique(),
                dataset=dataset,
                k=10,
                filter_viewed=True
            )

In [278]:
pop_recs_for_all_users = pop_recs_for_all_users.groupby(['user_id']).agg({'item_id': lambda x: x.tolist()}).reset_index().set_index('user_id').to_dict()['item_id']

### Напишем функцию для того, чтобы объединить рекомендации UserKnn() и PopularModel() по тому же принципу
(Если `UserKnn()` предлагает < 10 айтемов, то дополняем самыми популярными до 10)

In [239]:
def calc_offline_recs(knn_recs, 
                      pop_recs):
    final_offline_recs = {}
    for user in interactions_df['user_id'].unique():
        if user in knn_recs:
            cur_user_knn_recs = knn_recs[user]
        else:
            
            cur_user_knn_recs = []
    
        if len(cur_user_knn_recs) < 10:
            final_offline_recs[user] = pd.unique(np.concatenate([cur_user_knn_recs, 
                                                                pop_recs[user]]))[:10].tolist()
        else:
            final_offline_recs[user] = cur_user_knn_recs
    
    return final_offline_recs

In [240]:
final_offline_recs = calc_offline_recs(knn_recs_for_all_users,
                                      pop_recs_for_all_users)

### Сохраним это в пикл

In [247]:
with open("../service/recsys_models/knn_tfidf_model_offline_recos_20231129.pkl", "wb") as f:
    pickle.dump(final_offline_recs, f)

### Однако бот запрашивает рекомендации и для тех, кого нет в датасете (соответственно и в файле с оффлайн рекомендациями)
- Тогда мы вычислим вручную 10 самых популярных айтемов и будем предлагать их абсолютно новым пользователям
- Мы вновь завернем эти айтемы в пикл, и будем обращаться к нему с сервака

In [298]:
pop_model.recommend(
                users=interactions_df['user_id'].unique(),
                dataset=dataset,
                k=10,
                filter_viewed=True
            ).groupby(['user_id']).agg({'item_id': lambda x: x.tolist()}).reset_index().set_index('user_id')['item_id'].value_counts().iloc[:1]

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]    398600
Name: item_id, dtype: int64

In [299]:
interactions_df['item_id'].value_counts().iloc[:10].index.tolist()

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

### Мы попытались найти 10 самых популярных айтемов двумя способами
- Посчитав какая комбинация из 10 айтемов рекомендуется пользователям чаще всего
- Посчитав айтемы с которыми больше всего взаимодействовали в исходном датасете  

  
- В обоих случаях 10 айтемов оказались одинаковыми, ими и восползуемся при прогнозе

In [301]:
with open("../service/recsys_models/10_most_popular_items_20231129.pkl", "wb") as f:
    pickle.dump(interactions_df['item_id'].value_counts().iloc[:10].index.tolist(), f)

## Теперь у нас есть все необходимые пиклы для теста